In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import pandas as pd
import nltk
import re
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import re

#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import os
root_path = os.getcwd()
root_path

"C:\\Users\\Abhishek's PC\\Desktop\\datasets\\HACKATHON-Text to key_words"

In [4]:
import os
glove_dir = 'glove'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [5]:

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance


In [6]:
def read(data):
    file_name = open(data, "r")
    file = file_name.readlines()
#     print(file)
    for i in range(len(file)):
        text= file[i].split(". ")
        sentences = []
#         for i,sent in text:
        for sentence in text:
            sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
#             print(sentence)
        sentences.pop() 
    
    return sentences

    

In [8]:

def vectorizer(summ_data,embeddings_index):
    summ_vec={}
    count=1
    for list_w in summ_data:
        temp=[]
        for word in list_w:
            if word in embeddings_index.keys():
                temp.append(embeddings_index[word])
        summ_vec[count]=temp
        count=count+1
    return summ_vec


In [9]:
def word_binder_dict(vec):
    summ_sent_vec={}
    for num,list_w in vec.items():
        n=1
        temp=np.zeros(200)
        for word in list_w:
            temp=temp+word
            n=n+1
        summ_sent_vec[num]=(temp/n)
    return summ_sent_vec

In [10]:
def doc_binder(vec):
    temp=np.zeros(200)
    n=1
    for num,list_w in vec.items():
        temp=temp+list_w
        n=n+1
    doc_vec=(temp/n)
        
    return doc_vec

In [11]:
def dist_bw_sent_doc_norm2(vec1,vec2):
    dist_arr={}
    for num,sent in vec1.items():
        dist = np.linalg.norm(vec2-sent)
        dist_arr[num]=dist
    return dist_arr

In [12]:
def dist_bw_sent_doc_cos(vec1,vec2):
    dist_arr={}
    for num,sent in vec1.items():
        dist =cosine_distance(vec2,sent)
        dist_arr[num]=dist
    return dist_arr

In [13]:
def summarizer(doc_norm2_dist,joined_summ):
    summarize_text = []
    meaner=[]
    for i in range(1,len(doc_norm2_dist)+1):
        meaner.append(doc_norm2_dist[i])
    mean_arr=np.array(meaner)
    threshold=((mean_arr.mean()+np.median(mean_arr))/2)
    for i in range(1,len(doc_norm2_dist)+1):
        if doc_norm2_dist[i]<threshold:
             summarize_text.append("".join(joined_summ[i]))
    return summarize_text


In [14]:
# stop_words = set(stopwords.words("english"))

nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
def corpus_maker(summarize_text):
    corpus = []
    for i in range(len(summarize_text)):
        #Remove punctuations
        text = re.sub('[^a-zA-Z]', ' ',summarize_text[i] )

        #Convert to lowercase
        text = text.lower()

        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)

        ##Convert to list from string
        text = text.split()

        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
                stop_words] 
        text = " ".join(text)
        corpus.append(text)
    return corpus 


[nltk_data] Downloading package wordnet to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]


In [16]:
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2),  
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]


In [17]:
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3), 
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]


In [27]:

def generate_summ_key(path):
    summ_data=read(path)
    joined_summ={}
    n=0
    
    for list_w in summ_data:
        n=n+1
        text=' '.join(list_w)
        joined_summ[n]=text
        
    summ_vec=vectorizer(summ_data,embeddings_index)
    summ_sent_vec=word_binder_dict(summ_vec)
    doc_vec=doc_binder(summ_sent_vec)
    doc_cos_dist=dist_bw_sent_doc_cos(summ_sent_vec,doc_vec)
    doc_norm2_dist=dist_bw_sent_doc_norm2(summ_sent_vec,doc_vec)
    
    summarize_text=summarizer(doc_norm2_dist,joined_summ)
    with open('summary.txt', 'w') as f:
        for item in summarize_text:
            f.write("%s\n" % item)
    
    stop_words = set(stopwords.words("english"))
        
    corpus=corpus_maker(summarize_text)
    cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
    X=cv.fit_transform(corpus)
    
    n=5
    
    top_words = get_top_n_words(corpus, n)
    top_df = pd.DataFrame(top_words)
    top_df.columns=["Word", "Freq"]
    
    keyword=[]
    
    for word in top_df['Word']:
        keyword.append(word)
    
    top2_words = get_top_n2_words(corpus, n)
    top2_df = pd.DataFrame(top2_words)
    top2_df.columns=["Bi-gram", "Freq"]
    top2_df['Bi-gram']
    
    bi=[]
    
    for word in top2_df['Bi-gram']:
        keyword.append(word)
   
    top3_words = get_top_n3_words(corpus, n)
    top3_df = pd.DataFrame(top3_words)
    top3_df.columns=["Tri-gram", "Freq"]
    
    tri=[]
    
    for word in top3_df['Tri-gram']:
        keyword.append(word)
    with open('keyword.txt', 'w') as f:
        for item in keyword:
            f.write("%s\n" % item)
generate_summ_key("help_data.txt")